In [19]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///D:/bld/scikit-learn_1708073997094/work


In [25]:
!pip list --format=freeze | grep scikit-learn

scikit-learn==1.5.0


In [22]:
!python -V

Python 3.12.2


In [23]:
import pickle
import pandas as pd

In [26]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [27]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [28]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [29]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [30]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [31]:
y_pred.std()

6.247488852238703

In [32]:
output_file = '../output/yellow_tripdata_2023-03.parquet'
year = 2023
month = 3

In [33]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [34]:
df['predicted_duration'] = y_pred

In [35]:
df_result = df[['ride_id', 'predicted_duration']]

In [36]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [38]:
!ls -la ../output

total 67036
drwxr-xr-x 1 kiramishima 197609        0 Jun 15 19:23 .
drwxr-xr-x 1 kiramishima 197609        0 Jun 15 18:45 ..
-rw-r--r-- 1 kiramishima 197609 68641880 Jun 15 21:19 yellow_tripdata_2023-03.parquet
